In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from user_agents import parse

In [2]:
spark = SparkSession.builder.appName('4_1').master('local[2]').config('driver.memory', '8g').getOrCreate()

In [3]:
rdd = spark.sparkContext.textFile('../dataset1')

In [4]:
rdd.count()

1977

In [5]:
def clean_window_os(user_agent):
    if user_agent.os.family.startswith('Windows'):
        return 'Windows'
    else:
        return user_agent.os.family

def clean_window_os_version(user_agent):
    if user_agent.os.family.startswith('Windows'):
        return user_agent.os.family.split(' ')[1].split('.')[0]
    elif user_agent.os.family.startswith('Mac OS X'):
        tmp=user_agent.os.version_string.split('.')[:2]
        return '.'.join(tmp)
    elif not user_agent.os.version_string:
        return '-1'
    else:
        return user_agent.os.version_string.split('.')[0]

def clean_browser(user_agent):
    if user_agent.browser.family.startswith('Chrome Mobile'):
        return 'Chrome'
    elif user_agent.browser.family.startswith('Mobile Safari'):
        return 'Safari'
    elif user_agent.browser.family.startswith('Opera Mobile'):
        return 'Opera'
    else:
        return user_agent.browser.family

In [6]:
df = rdd\
    .map(lambda string: parse(string))\
    .map(lambda user_agent: Row(
        browser=clean_browser(user_agent),
        browser_version=user_agent.browser.version_string.split('.')[0],
        os=clean_window_os(user_agent),
        os_version=clean_window_os_version(user_agent),
        device_brand=user_agent.device.brand,
        device_model=user_agent.device.model,
        is_mobile=int(user_agent.is_mobile),
        is_tablet=int(user_agent.is_tablet),
        is_pc=int(user_agent.is_pc),
        is_bot=int(user_agent.is_bot)
    )).toDF()\
    .na.fill({'device_brand': 'Other', 'device_model': 'Other', 'os_version': '-1'})\
    .cache()
df.show(5)

+----------------+---------------+------------+------------+------+---------+-----+---------+-------+----------+
|         browser|browser_version|device_brand|device_model|is_bot|is_mobile|is_pc|is_tablet|     os|os_version|
+----------------+---------------+------------+------------+------+---------+-----+---------+-------+----------+
|         Firefox|             54|       Other|       Other|     0|        0|    1|        0|Windows|        10|
|Samsung Internet|              5|     Samsung|    SM-G935F|     0|        1|    0|        0|Android|         7|
|          Chrome|             59|       Other|       Other|     0|        0|    1|        0|Windows|         8|
|          Chrome|             59|       Other|       Other|     0|        0|    1|        0|Windows|         7|
|          Safari|             10|       Apple|      iPhone|     0|        1|    0|        0|    iOS|        10|
+----------------+---------------+------------+------------+------+---------+-----+---------+---

In [7]:
df.printSchema()

root
 |-- browser: string (nullable = true)
 |-- browser_version: string (nullable = true)
 |-- device_brand: string (nullable = false)
 |-- device_model: string (nullable = false)
 |-- is_bot: long (nullable = true)
 |-- is_mobile: long (nullable = true)
 |-- is_pc: long (nullable = true)
 |-- is_tablet: long (nullable = true)
 |-- os: string (nullable = true)
 |-- os_version: string (nullable = false)



In [8]:
#  CFNetwork|        1|    1|        0|     0|
#  Chrome|        0|    0|        0|     0|
#  Googlebot|        1|    0|        0|     1|
#  Puffin|        1|    1|        0|     0|
#  Safari|        1|    0|        1|     0|
df.where(
    '''
    (browser='CFNetwork' and is_mobile=1 and is_pc=1 and is_tablet=0 and is_bot=0) or
    (browser='Chrome' and is_mobile=0 and is_pc=0 and is_tablet=0 and is_bot=0) or
    (browser='Googlebot' and is_mobile=1 and is_pc=0 and is_tablet=0 and is_bot=1) or
    (browser='Puffin' and is_mobile=1 and is_pc=1 and is_tablet=0 and is_bot=0) or
    (browser='Safari' and is_mobile=1 and is_pc=0 and is_tablet=1 and is_bot=0)
    ''').show()
# following are weird.

+---------+---------------+------------+------------+------+---------+-----+---------+-------+----------+
|  browser|browser_version|device_brand|device_model|is_bot|is_mobile|is_pc|is_tablet|     os|os_version|
+---------+---------------+------------+------------+------+---------+-----+---------+-------+----------+
|   Safari|              4|     Generic|  Smartphone|     0|        1|    0|        1|Android|        -1|
|   Chrome|             41|       Other|       Other|     0|        0|    0|        0|  Other|        -1|
|CFNetwork|            811|       Apple|  iOS-Device|     0|        1|    1|        0|Windows|         7|
|Googlebot|              2|      Spider|  Smartphone|     1|        1|    0|        0|Android|         6|
|Googlebot|              2|      Spider|  Smartphone|     1|        1|    0|        0|Android|         6|
|   Puffin|              6|     Generic|  Smartphone|     0|        1|    1|        0|  Linux|        -1|
+---------+---------------+------------+------

In [9]:
df.select('browser').where('is_bot = false').groupby('browser').count().sort(desc('count')).show(5)
print('total: {}'.format(df.select('browser').where('is_bot = false').count()))

+----------------+-----+
|         browser|count|
+----------------+-----+
|          Chrome|  881|
|          Safari|  376|
|        Facebook|  236|
|              IE|  115|
|Samsung Internet|   39|
+----------------+-----+
only showing top 5 rows

total: 1745


In [10]:
df.select('browser').where('is_bot = true').groupby('browser').count().sort(desc('count')).show(5)
print('total: {}'.format(df.select('browser').where('is_bot = true').count()))

+-------------+-----+
|      browser|count|
+-------------+-----+
|    Googlebot|  218|
|    YandexBot|    7|
| Yahoo! Slurp|    4|
|AdsBot-Google|    2|
|       okhttp|    1|
+-------------+-----+

total: 232


In [11]:
df.where('is_pc = true')\
    .rollup('browser', 'browser_version').count().dropna().sort(desc('count')).show(5)

+-------+---------------+-----+
|browser|browser_version|count|
+-------+---------------+-----+
| Chrome|             59|  339|
|     IE|             11|  103|
| Chrome|             49|   39|
| Chrome|             60|   32|
|Firefox|             54|   23|
+-------+---------------+-----+
only showing top 5 rows



In [12]:
df.where('is_mobile = true')\
    .rollup('browser', 'browser_version').count().dropna().sort(desc('count')).show(5)

+----------------+---------------+-----+
|         browser|browser_version|count|
+----------------+---------------+-----+
|          Safari|             10|  285|
|          Chrome|             59|  260|
|        Facebook|            134|  133|
|          Chrome|             58|   35|
|Samsung Internet|              5|   29|
+----------------+---------------+-----+
only showing top 5 rows



In [13]:
df.where('is_mobile = true and os = "Android"')\
    .select('device_brand')\
    .groupby('device_brand')\
    .count()\
    .sort(desc('count'))\
    .show(10)

+---------------+-----+
|   device_brand|count|
+---------------+-----+
|        Samsung|  224|
|           Asus|   85|
|Generic_Android|   84|
|            HTC|   82|
|         XiaoMi|   28|
|   SonyEricsson|   22|
|           Sony|   13|
|             LG|   12|
|           Oppo|    9|
|           vivo|    5|
+---------------+-----+
only showing top 10 rows



In [14]:
df.where('is_mobile = true')\
    .select('os')\
    .groupby('os')\
    .count()\
    .sort(desc('count'))\
    .show(5)

+-------+-----+
|     os|count|
+-------+-----+
|Android|  572|
|    iOS|  457|
|Windows|    1|
|  Linux|    1|
+-------+-----+



In [15]:
df.where('is_pc = true')\
    .select('os')\
    .groupby('os')\
    .count()\
    .sort(desc('count'))\
    .show(5)

+--------+-----+
|      os|count|
+--------+-----+
| Windows|  608|
|Mac OS X|   26|
|   Linux|    8|
|  Ubuntu|    2|
+--------+-----+



In [16]:
df.toPandas().to_csv('dataset1.csv', index=False, header=True, encoding='utf-8')

<div>
    <a href="https://plot.ly/~avidoggy/5/?share_key=EzvdrKsYvsk1M09Q5tzOAE" target="_blank" title="Plot 5" style="display: block; text-align: center;"><img src="https://plot.ly/~avidoggy/5.png?share_key=EzvdrKsYvsk1M09Q5tzOAE" alt="Plot 5" style="max-width: 100%;width: 600px;"  width="600" onerror="this.onerror=null;this.src='https://plot.ly/404.png';" /></a>
    <script data-plotly="avidoggy:5" sharekey-plotly="EzvdrKsYvsk1M09Q5tzOAE" src="https://plot.ly/embed.js" async></script>
</div>